In [1]:
job_description = '''Sto gestendo una posizione per una figura di Team Leader per conto di una realtà finale ultra-innovativa, specializzata nello sviluppo di prodotti e soluzioni proprietarie nell'ambito data science, machine learning, math modeling development ed intelligenza artificiale.
'''
with open("full_experience.txt", "r") as file:
    actual_experience = file.read()
with open("example_profile.txt", "r") as file:
    example = file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'full_experience.txt'

In [ ]:
system_prompt = '''You are a resume expert and you will write the section of a resume based on actual experience and tuned for the job description that the user asks.
You will produce the following outputs:
#PROFILE: high level presentation of candidate tuned for the job description (max 100 words).
#MANAGER PROJECTS: select and rephrase main projects inherent for the job description among the manager ones (max 3 points).
#SENIOR DATA SCIENTIST PROJECTS: select and rephrase main projects inherent for the job description among the senior data scientist ones (max 3 points).
#RESEARCH FELLOW PROJECTS: select and rephrase main projects inherent for the job description among the research fellow ones (max 3 points).
#KEY WORDS: key words related to actual experience and tuned for the job description (max 5 keywords).

The candidate's actual experience is the following:
{}

Note: Do not invet informations, just rephrase actual experience to meet the job description. Create the output in English.
'''.format(actual_experience)
print(system_prompt)
assistant_prompt = '''Here an example of the desired output:
{}
'''.format(example)
print(assistant_prompt)
user_prompt = '''
Write the candidate's resume in English for the following job description:
{}
'''.format(job_description)
print(user_prompt)

You are a resume expert and you will write the section of a resume based on actual experience and tuned for the job description that the user asks.
You will produce the following outputs:
#PROFILE: high level presentation of candidate tuned for the job description (max 100 words).
#MANAGER PROJECTS: select and rephrase main projects inherent for the job description among the manager ones (max 3 points).
#SENIOR DATA SCIENTIST PROJECTS: select and rephrase main projects inherent for the job description among the senior data scientist ones (max 3 points).
#RESEARCH FELLOW PROJECTS: select and rephrase main projects inherent for the job description among the research fellow ones (max 3 points).
#KEY WORDS: key words related to actual experience and tuned for the job description (max 5 keywords).

The candidate's actual experience is the following:
profile:
formation: PhD cum Laude in Nuclear and Energy Science and Engineering, Master degree in Nuclear Engineering 
known technical instrume

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "assistant", "content": assistant_prompt},
    {"role": "user", "content": user_prompt}
]
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=0.5,
  max_tokens=1024
)
output = response['choices'][0]['message']['content']

print(output)

#PROFILE: 
Highly skilled and experienced Team Leader with a strong background in data science, machine learning, math modeling development, and artificial intelligence. Proven track record in managing teams and delivering innovative solutions. 

#MANAGER PROJECTS:
* Led the development of a custom ChatBot for a leading newspaper, utilizing state-of-the-art language models (OpenAI, HuggingFace).
* Successfully implemented anomaly detection for the telecommunications industry using probabilistic recurrent neural networks.
* Managed the HR advanced analytics project for an Energy & Utilities society, leveraging large language models for the classification of misused contracts.

#SENIOR DATA SCIENTIST PROJECTS:
* Conducted self-supervised pattern extraction from customers of a trading platform, employing multi-modal multi-task variational autoencoders.
* Defined data science guidelines (coding, MLOps, governance) for the Energy & Utilities industry.
* Developed long-term budget forecastin

In [ ]:
output

'#PROFILE: \nHighly skilled and experienced Team Leader with a strong background in data science, machine learning, math modeling development, and artificial intelligence. Proven track record in managing teams and delivering innovative solutions. \n\n###* Led the development of a custom ChatBot for a leading newspaper, utilizing state-of-the-art language models (OpenAI, HuggingFace).\n* Successfully implemented anomaly detection for the telecommunications industry using probabilistic recurrent neural networks.\n* Managed the HR advanced analytics project for an Energy & Utilities society, leveraging large language models for the classification of misused contracts.\n\n###* Conducted self-supervised pattern extraction from customers of a trading platform, employing multi-modal multi-task variational autoencoders.\n* Defined data science guidelines (coding, MLOps, governance) for the Energy & Utilities industry.\n* Developed long-term budget forecasting and industrialization on cloud pla

In [ ]:
splits = ['#PROFILE:', '#MANAGER PROJECTS:', '#SENIOR DATA SCIENTIST PROJECTS:', 
    '#RESEARCH FELLOW PROJECTS:', '#KEY WORDS:']
for split in splits:
    output = output.replace(split, '###')
output_list = output.split('###')[1:]

out_dict = {}
for split, o in zip(splits, output_list):
    section_name = split.replace(':', '').replace(' ', '').replace('\n', '')
    if o[0] == '*':
        subsplits = o.split('* ')[1:]
        for i, subsplit in enumerate(subsplits):
            out_dict[section_name+str(i+1)] = subsplit.replace('\n', '')
    else:
        out_dict[section_name] = o.replace('\n', '').lstrip()

In [ ]:
out_dict

{'#PROFILE': 'Highly skilled and experienced Team Leader with a strong background in data science, machine learning, math modeling development, and artificial intelligence. Proven track record in managing teams and delivering innovative solutions. ',
 '#MANAGERPROJECTS1': 'Led the development of a custom ChatBot for a leading newspaper, utilizing state-of-the-art language models (OpenAI, HuggingFace).',
 '#MANAGERPROJECTS2': 'Successfully implemented anomaly detection for the telecommunications industry using probabilistic recurrent neural networks.',
 '#MANAGERPROJECTS3': 'Managed the HR advanced analytics project for an Energy & Utilities society, leveraging large language models for the classification of misused contracts.',
 '#SENIORDATASCIENTISTPROJECTS1': 'Conducted self-supervised pattern extraction from customers of a trading platform, employing multi-modal multi-task variational autoencoders.',
 '#SENIORDATASCIENTISTPROJECTS2': 'Defined data science guidelines (coding, MLOps, 

In [ ]:
from docx import Document
from datetime import date
import re


def docx_replace_regex(doc_obj, regex, replace):

    for p in doc_obj.paragraphs:
        if regex in p.text:
            p.text = p.text.replace(regex, replace)

    # for obj in doc_obj.inline_shapes:
    #     if regex in obj.text:
    #         obj.text = obj.text.replace(regex, replace)

    for table in doc_obj.tables:
        for row in table.rows:
            for cell in row.cells:
                docx_replace_regex(cell, regex, replace)

document = Document('CV_APazzaglia_template.docx')
for key in out_dict:
    replace_text = out_dict[key]
    docx_replace_regex(document, key, replace_text)

document.save('outputs/CV_APazzaglia_{}.docx'.format(date.today()))